In [1]:
from IPython.lib.deepreload import original_import
%load_ext autoreload
%autoreload 2

import ex4
import numpy as np
import matplotlib.pyplot as plt

### Example of a test script

In [3]:
import modular_ex4
import imageio
import matplotlib.pyplot as plt

# ----------------- הגדרות ניסוי -----------------
VIDEO_FILE = "House.mp4"
STEP_SIZE = 16        # פרמטר לשליטה
BORDER_CUT = 15       # פרמטר לשליטה
ENABLE_ROT = True     # עם או בלי תיקון רוטציה
CONV_POINT = None     # אופציונלי: (x,y)
# CONV_POINT = (300, 250) # לדוגמה, אם אתה רוצה למרכז אובייקט

# ----------------- ביצוע ה-Pipeline -----------------

# 1. טעינה (פעם אחת)
raw_frames = modular_ex4.load_video_frames(VIDEO_FILE, downscale_factor=2)
h, w = raw_frames[0].shape[:2]

# 2. ייצוב (פעם אחת, לוקח זמן)
# מעבירים כאן את הפרמטרים כי הייצוב מחשב optical_flow
stable_frames = modular_ex4.stabilize_video(
    raw_frames,
    step_size=STEP_SIZE,
    border_cut=BORDER_CUT,
    enable_rotation=ENABLE_ROT
)

# 3. חישוב נתיב וגיאומטריה (מהיר)
# גם כאן מעבירים את הפרמטרים כי הוא מחשב שוב optical_flow על המיוצב
transforms = modular_ex4.compute_camera_path(
    stable_frames,
    step_size=STEP_SIZE,
    border_cut=BORDER_CUT,
    convergence_point=CONV_POINT
)

canvas_geo = modular_ex4.compute_canvas_geometry(transforms, h, w)

# 4. יצירת סרטון פנורמה (Multi-Perspective)
# כאן מייצרים פנורמות שונות על ידי שינוי ה-Anchor
print("Rendering multi-perspective movie...")
movie_frames = []
anchors = np.linspace(0, 1, num=20) # 20 פריימים לסרטון

for anchor in anchors:
    # שלב הרינדור לא צריך step_size כי אין בו חישוב תנועה, רק גזירה
    pan = modular_ex4.render_strip_panorama(stable_frames, transforms, canvas_geo, strip_anchor=anchor)

    # המרה ל-uint8 לשמירה בוידאו
    pan_uint8 = (np.clip(pan, 0, 1) * 255).astype(np.uint8)
    movie_frames.append(pan_uint8)

# שמירת התוצאה
imageio.mimsave("outputs/multi_perspective_output.mp4", movie_frames, fps=10)
print("Done! Video saved.")

Loading video: House.mp4...
Step 1: Stabilizing Video (Rotation=ON)...
Step 2: Computing Camera Path...
Step 3: Calculating Canvas Geometry...
  > Canvas: 2249x968, Offset: (1608.2, 607.9)
Rendering multi-perspective movie...
Step 4: Rendering (Anchor=0.0, Padding=2, Gray=False)...
Step 4: Rendering (Anchor=0.05263157894736842, Padding=2, Gray=False)...
Step 4: Rendering (Anchor=0.10526315789473684, Padding=2, Gray=False)...
Step 4: Rendering (Anchor=0.15789473684210525, Padding=2, Gray=False)...
Step 4: Rendering (Anchor=0.21052631578947367, Padding=2, Gray=False)...
Step 4: Rendering (Anchor=0.2631578947368421, Padding=2, Gray=False)...
Step 4: Rendering (Anchor=0.3157894736842105, Padding=2, Gray=False)...
Step 4: Rendering (Anchor=0.3684210526315789, Padding=2, Gray=False)...
Step 4: Rendering (Anchor=0.42105263157894735, Padding=2, Gray=False)...
Step 4: Rendering (Anchor=0.47368421052631576, Padding=2, Gray=False)...
Step 4: Rendering (Anchor=0.5263157894736842, Padding=2, Gray=F

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (2249, 968) to (2256, 976) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Done! Video saved.


In [4]:
# שמירת תמונה אחת לדוגמה (מהמרכז)
modular_ex4.save_panorama(movie_frames[10], filename="center_view.jpg")

Saving to: outputs/center_view.jpg


### lab

In [ ]:
# garden_video="Garden.mp4"
house_video="House.mp4"
frames = ex4.load_video_frames(house_video, downscale_factor=2)
panorama = ex4.create_mosaic(frames, step_size=16, border_cut=15)
print("done.")

In [ ]:
plt.figure(figsize=(12, 6))
plt.imshow(panorama, cmap='gray')
plt.title("Grayscale Panorama")
plt.show()

In [ ]:
ex4.save_panorama(panorama, filename="house_panorama_gray_wo_rot.png")

In [ ]:
def plot_camera_path(frames, step_size, border_cut):
    thetas = [0]
    cumulative_theta = 0

    print("Analyzing camera path...")
    for i in range(len(frames) - 1):
        _, _, theta = ex4.optical_flow(frames[i], frames[i+1], step_size, border_cut)
        cumulative_theta += theta
        thetas.append(np.rad2deg(cumulative_theta))

    plt.figure(figsize=(10, 4))
    plt.plot(thetas, label='Camera Rotation (Degrees)')
    plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    plt.title("Accumulated Rotation over Time")
    plt.xlabel("Frame Index")
    plt.ylabel("Angle (Degrees)")
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f"Total Rotation: {thetas[-1]:.2f} degrees")
    return thetas

plot_camera_path(frames, step_size=16, border_cut=15)